<a href="https://colab.research.google.com/github/anindo78/hugging_face_agents-course/blob/main/multiagent_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.8/149.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 16.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `WebSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W),
and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [6]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [7]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [8]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W),                                                                             │
│ and return them to me as a pandas dataframe.                                                                    │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: Millennium Mills, Royal Victoria Dock, East London. The Batman location: Lt Gordon and
The Batman interrogate Penguin after the ...

1. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 22 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

2. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) · 87 ; St.
George's Hall, Liverpool, England, UK. ( ...

3. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqO1a9a6DCPjtoqkagyCGMmWfCfidVOIhMxcrj
zLDtWldYS2Adt)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

6. [Locations | 1966 Batman Pages](https://www.66batmania.com/guides/locations/)
Source: 66batmania.com

The actual filming of the series was predominately undertaken at 'The Culver City Studios'. Other notable filming 
locations include; 'The Warner Brothers ...

7. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

8. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

9. [Tag: Batman filming locations](https://www.iamnotastalker.com/tag/batman-filming-locations/)
Source: IAMNOTASTALKER

Shooting on location was a very small, independent caveman/apocalypse film. And the crew was even nice enough to 
let us snap some pics of them.

Out: None

[Step 1: Duration 8.85 seconds| Input tokens: 2,324 | Output tokens: 151]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extract supercar factories and their coordinates                                                               
  supercar_factories = [                                                                                           
      ("Lamborghini Museum in Funo di Argelato, Bologna, Italy", (44.6426, 11.2182)),                              
      ("Lamborghini Museum 'MUDETEC' in Sant'Agata Bolognese, Italy", (44.6426, 11.2182)),                         
      ("Lamborghini Factory in Sant'Agata Bolognese, Italy", (44.6426, 11.2182)),                                  
      ("Ferrari Gallery in Maranello, Italy", (44.5646, 10.9941)),                                                 
      ("Pagani Automobili in San Cesario sul Panaro, Italy", (44.6500, 10.9833)),                                  
      ("Maserati Museum in Maranello, Italy", (44.5646, 10.9941)),                                                 
      ("Ducati Museum in Bologna, Italy", (44.5055, 11.3364)),                                                     
      ("Koenigsegg HQ, Angelholm, Sweden", (56.1589, 12.8906))                                                     
  ]                                                                                                                
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = []                                                                                       
  for location, coords in supercar_factories:                                                                      
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times.append((location, travel_time))                                                        
                                                                                                                   
  # Create DataFrame for supercar factories                                                                        
  supercar_df = pd.DataFrame(supercar_travel_times, columns=['Location', 'Travel Time (hours)'])                   
                                                                                                                   
  # Print Supercar DataFrame                                                                                       
  print(supercar_df)                                                                                               
                                                                                                                   
  # Combine both DataFrames                                                                                        
  combined_df = pd.concat([batman_df, supercar_df], ignore_index=True)                                             
                                                                                                                   
  # Print Combined DataFrame                                                                                       
  print(combined_df)                                                                                               
                                                                                                                   
  # Return the final answer                                                                                        
  final_answer(combined_df)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                            Location  Travel Time (hours)
0  Lamborghini Museum in Funo di Argelato, Bologn...                10.75
1  Lamborghini Museum 'MUDETEC' in Sant'Agata Bol...                10.75
2  Lamborghini Factory in Sant'Agata Bolognese, I...                10.75
3                Ferrari Gallery in Maranello, Italy                10.73
4  Pagani Automobili in San Cesario sul Panaro, I...                10.73
5                Maserati Museum in Maranello, Italy                10.73
6                    Ducati Museum in Bologna, Italy                10.77
7                   Koenigsegg HQ, Angelholm, Sweden                10.07
                                             Location  Travel Time (hours)
0   Millennium Mills, Royal Victoria Dock, East Lo...                 9.18
1   Knebworth House, Knebworth, Hertfordshire, Eng...                 9.14
2   Acton Lane Power Station, Acton Lane, Acton, L...                 9.16
3          Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
4           St. George's Hall, Liverpool, England, UK                 8.81
5                   Cliffs of Marineland, Los Angeles                 6.81
6                         Kirkeby Center, Los Angeles                 6.81
7                  Gateway East Building, Los Angeles                 6.81
8                            Wayne Manor, Los Angeles                 6.81
9                          Bronson Caves, Los Angeles                 6.81
10  James R Thompson Center, West Randolph Street,...                 2.68
11  Lamborghini Museum in Funo di Argelato, Bologn...                10.75
12  Lamborghini Museum 'MUDETEC' in Sant'Agata Bol...                10.75
13  Lamborghini Factory in Sant'Agata Bolognese, I...                10.75
14                Ferrari Gallery in Maranello, Italy                10.73
15  Pagani Automobili in San Cesario sul Panaro, I...                10.73
16                Maserati Museum in Maranello, Italy                10.73
17                    Ducati Museum in Bologna, Italy                10.77
18                   Koenigsegg HQ, Angelholm, Sweden                10.07

Final answer:                                              Location  Travel Time (hours)
0   Millennium Mills, Royal Victoria Dock, East Lo...                 9.18
1   Knebworth House, Knebworth, Hertfordshire, Eng...                 9.14
2   Acton Lane Power Station, Acton Lane, Acton, L...                 9.16
3          Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
4           St. George's Hall, Liverpool, England, UK                 8.81
5                   Cliffs of Marineland, Los Angeles                 6.81
6                         Kirkeby Center, Los Angeles                 6.81
7                  Gateway East Building, Los Angeles                 6.81
8                            Wayne Manor, Los Angeles                 6.81
9                          Bronson Caves, Los Angeles                 6.81
10  James R Thompson Center, West Randolph Street,...                 2.68
11  Lamborghini Museum in Funo di Argelato, Bologn...                10.75
12  Lamborghini Museum 'MUDETEC' in Sant'Agata Bol...                10.75
13  Lamborghini Factory in Sant'Agata Bolognese, I...                10.75
14                Ferrari Gallery in Maranello, Italy                10.73
15  Pagani Automobili in San Cesario sul Panaro, I...                10.73
16                Maserati Museum in Maranello, Italy                10.73
17                    Ducati Museum in Bologna, Italy                10.77
18                   Koenigsegg HQ, Angelholm, Sweden                10.07

[Step 3: Duration 5.86 seconds| Input tokens: 11,264 | Output tokens: 1,239]

In [9]:
result

,Location,Travel Time (hours)
0,"Millennium Mills, Royal Victoria Dock, East Lo...",9.18
1,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.14
2,"Acton Lane Power Station, Acton Lane, Acton, L...",9.16
3,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
4,"St. George's Hall, Liverpool, England, UK",8.81
5,"Cliffs of Marineland, Los Angeles",6.81
6,"Kirkeby Center, Los Angeles",6.81
7,"Gateway East Building, Los Angeles",6.81
8,"Wayne Manor, Los Angeles",6.81
9,"Bronson Caves, Los Angeles",6.81


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [15]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W),                                                                             │
│ and return them to me as a pandas dataframe.                                                                    │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We need to find Batman filming locations worldwide.
- We need to calculate the travel time for a cargo plane to Gotham for each filming location.
- We need to find supercar factories with the same cargo plane transfer time to Gotham.

### 1.2. Facts to look up
- List of Batman filming locations worldwide.
  - Source: Web search for "Batman filming locations" and verify from reliable sources like IMDb, official movie 
websites, or reputable news articles.
- Coordinates of each Batman filming location.
  - Source: Geocoding the addresses of the filming locations using a geocoding API or manually from maps.
- List of supercar factories worldwide.
  - Source: Web search for "supercar factories" and verify from reliable sources like automotive news websites or 
official manufacturer websites.
- Coordinates of each supercar factory.
  - Source: Geocoding the addresses of the supercar factories using a geocoding API or manually from maps.

### 1.3. Facts to derive
- Travel time for a cargo plane from each Batman filming location to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the origin coordinates of each filming location
and the destination coordinates of Gotham.
- Travel time for a cargo plane from each supercar factory to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the origin coordinates of each supercar factory
and the destination coordinates of Gotham.

## 2. Plan
1. Perform a web search for "Batman filming locations" and gather a list of locations.
2. For each Batman filming location, perform a web search to find its coordinates.
3. Perform a web search for "supercar factories" and gather a list of factories.
4. For each supercar factory, perform a web search to find its coordinates.
5. Use the `calculate_cargo_travel_time` function to calculate the travel time from each Batman filming location to
Gotham.
6. Use the `calculate_cargo_travel_time` function to calculate the travel time from each supercar factory to 
Gotham.
7. Create a pandas dataframe with Batman filming locations and their travel times to Gotham.
8. Create a pandas dataframe with supercar factories and their travel times to Gotham.
9. Merge the two dataframes based on travel time to Gotham.
10. Finalize the dataframes and prepare them for the final answer.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Perform a web search for "Batman filming locations"                                                    
  batman_locations_query = "Batman filming locations"                                                              
  batman_locations_results = web_search(query=batman_locations_query)                                              
  print(batman_locations_results)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: Millennium Mills, Royal Victoria Dock, East London. The Batman location: Lt Gordon and
The Batman interrogate Penguin after the ...

1. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 22 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

2. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) · 87 ; St.
George's Hall, Liverpool, England, UK. ( ...

3. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqO1a9a6DCPjtoqkagyCGMmWfCfidVOIhMxcrj
zLDtWldYS2Adt)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

6. [Locations | 1966 Batman Pages](https://www.66batmania.com/guides/locations/)
Source: 66batmania.com

The actual filming of the series was predominately undertaken at 'The Culver City Studios'. Other notable filming 
locations include; 'The Warner Brothers ...

7. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

8. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

9. [Tag: Batman filming locations](https://www.iamnotastalker.com/tag/batman-filming-locations/)
Source: IAMNOTASTALKER

Shooting on location was a very small, independent caveman/apocalypse film. And the crew was even nice enough to 
let us snap some pics of them.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 4: Extract coordinates for each supercar factory                                                          
  supercar_factories = [                                                                                           
      "Ferrari, Maranello, Italy",                                                                                 
      "Lamborghini, Sant'Agata Bolognese, Italy",                                                                  
      "Pagani, San Cesario sul Panaro, Italy",                                                                     
      "Maserati, Maranello, Italy",                                                                                
      "Ducati, Bologna, Italy"                                                                                     
  ]                                                                                                                
                                                                                                                   
  supercar_coords = []                                                                                             
                                                                                                                   
  for factory in supercar_factories:                                                                               
      query = f"coordinates of {factory}"                                                                          
      result = web_search(query=query)                                                                             
      lat, lon = extract_coordinates(result)                                                                       
      if not lat or not lon:                                                                                       
          # Fallback to searching for coordinates directly                                                         
          lat_match = re.search(r'Latitude: (-?\d+\.\d+)', result)                                                 
          lon_match = re.search(r'Longitude: (-?\d+\.\d+)', result)                                                
          if lat_match and lon_match:                                                                              
              lat = lat_match.group(1)                                                                             
              lon = lon_match.group(1)                                                                             
      supercar_coords.append((factory, lat, lon))                                                                  
                                                                                                                   
  # Step 5: Calculate travel time for cargo planes from each supercar factory to Gotham                            
  supercar_travel_times = []                                                                                       
                                                                                                                   
  for factory, lat, lon in supercar_coords:                                                                        
      if lat and lon:                                                                                              
          origin_coords = (float(lat.split('°')[0]), -float(lon.split('°')[0]))                                    
          travel_time = calculate_cargo_travel_time(origin_coords=origin_coords,                                   
  destination_coords=gotham_coords)                                                                                
          supercar_travel_times.append((factory, travel_time))                                                     
                                                         

Final answer:                                             Location  Travel Time to Gotham
0  Acton Lane Power Station, Acton Lane, Acton, L...                   9.21
1         Necropolis Cemetery, Glasgow, Scotland, UK                   9.35
2         Cliffs of Marineland, Los Angeles, CA, USA                  18.02
3               Kirkeby Center, Los Angeles, CA, USA                  18.02
4        Gateway East Building, Los Angeles, CA, USA                   8.22
5           Lamborghini, Sant'Agata Bolognese, Italy                   8.38
6                             Ducati, Bologna, Italy                   8.37

[Step 6: Duration 16.42 seconds| Input tokens: 73,415 | Output tokens: 2,732]

                                            Location  Travel Time to Gotham
0  Acton Lane Power Station, Acton Lane, Acton, L...                   9.21
1         Necropolis Cemetery, Glasgow, Scotland, UK                   9.35
2         Cliffs of Marineland, Los Angeles, CA, USA                  18.02
3               Kirkeby Center, Los Angeles, CA, USA                  18.02
4        Gateway East Building, Los Angeles, CA, USA                   8.22
5           Lamborghini, Sant'Agata Bolognese, Italy                   8.38
6                             Ducati, Bologna, Italy                   8.37


In [16]:
detailed_report

,Location,Travel Time to Gotham
0,"Acton Lane Power Station, Acton Lane, Acton, L...",9.21
1,"Necropolis Cemetery, Glasgow, Scotland, UK",9.35
2,"Cliffs of Marineland, Los Angeles, CA, USA",18.02
3,"Kirkeby Center, Los Angeles, CA, USA",18.02
4,"Gateway East Building, Los Angeles, CA, USA",8.22
5,"Lamborghini, Sant'Agata Bolognese, Italy",8.38
6,"Ducati, Bologna, Italy",8.37


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [17]:
import os
from google.colab import userdata
from smolagents import CodeAgent, GoogleSearchTool, VisitWebpageTool, InferenceClientModel
from smolagents.utils import encode_image_base64, make_image_url

# Retrieve the SERPER_API_KEY from Colab secrets and set it as an environment variable
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=2048
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [18]:
import os
from google.colab import userdata

# Retrieve the SERPER_API_KEY from Colab secrets and set it as an environment variable
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [20]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [21]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [22]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ InferenceClientModel - deepseek-ai/DeepSeek-R1 ────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
<think>
We are given a task that involves several parts:
1. Find all Batman filming locations in the world.
2. Calculate the time to transfer via cargo plane from Gotham (40.7128° N, 74.0060° W) to each of these locations.
3. Also find some supercar factories that have the same cargo plane transfer time (i.e., the same travel time as 
one of the Batman filming locations) and we need at least 6 points in total (so if we have fewer than 6 Batman 
locations, we will need to find enough supercar factories to make up the total to 6).
4. Represent all these locations (Batman filming locations and supercar factories) on a spatial map as scatter 
points, with color depending on the travel time, and save it to saved_map.png.

We have a tool to calculate travel time between two points given their coordinates and a cruising speed (default 
750 km/h). We also have a web_agent to look up information.

Let's break down the facts we need:

### 1.1. Facts given in the task
- We are in Gotham: coordinates (40.7128, -74.0060) [Note: longitude is given as 74.0060° W, which is negative in 
decimal degrees].
- We need to use a cargo plane with a cruising speed (default 750 km/h) for the travel time calculation.

### 1.2. Facts to look up
We need to find:
1. List of Batman filming locations (with their coordinates). We don't have this, so we must look it up.
   - Source: We can use the web_agent to search for "Batman filming locations around the world" or similar. We need
to get a list of places and then their coordinates.

2. List of supercar factories (with their coordinates). We also don't have this.
   - Source: Use the web_agent to search for "supercar factories around the world" or "manufacturers of supercars 
and their factory locations". We need to get a list of factories and their locations.

However, note that the task says: "Also give me some supercar factories with the same cargo plane transfer time." 
This means we are to find supercar factories that have a travel time from Gotham that matches one of the travel 
times of the Batman filming locations. But note: we don't know the travel times until we compute them. So we must 
first get the Batman locations and compute their travel times, then for each distinct travel time (or perhaps for 
each travel time we want to find a supercar factory with the same travel time) we find supercar factories that have
that travel time. However, the task says "at least 6 points in total", meaning the total of Batman locations plus 
supercar factories should be at least 6. So if we have 3 Batman locations, we need at least 3 supercar factories 
(each with one of the travel times of the Batman locations) to make 6.

But note: it says "with the same cargo plane transfer time". So we are to match the travel times. However, it 
doesn't specify that we must have one supercar factory per distinct travel time. We could have multiple supercar 
factories for the same travel time? But the goal is to have at least 6 points. So we can do:

- Step 1: Get Batman filming locations and compute their travel times.
- Step 2: If we have at least 6 Batman locations, then we don't need supercar factories? But the task says "also 
give me some supercar factories", so we must include some even if we have 6 or more Batman locations? Actually, the
task says "at least 6 points in total", meaning the total of both. So if we have 6 Batman locations, we can just 
use them and then also find supercar factories that have the same travel times? But note: it says "also", so we 
must include supercar factories. However, the requirement is at least 6 points in total. So we can have, for 
example, 3 Batman locations and 3 supercar factories, or 4 and 2, etc.

But note: the task says "Also give me some supercar factories with the same ca

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
We are going to follow the plan step by step.                                                                      
                                                                                                                   
Step 1: Use the web_agent to get Batman filming locations with coordinates.                                        
   - We'll ask the web_agent to return a list of dictionaries with keys: 'name', 'latitude', 'longitude'           
                                                                                                                   
Step 2: For each Batman location, calculate the travel time from Gotham (40.7128, -74.0060) using                  
`calculate_cargo_travel_time`.                                                                                     
                                                                                                                   
Step 3: Use the web_agent to get supercar factory locations with coordinates.                                      
                                                                                                                   
Step 4: For each supercar factory, calculate the travel time from Gotham.                                          
                                                                                                                   
Step 5: Match supercar factories to Batman travel times with a tolerance. We'll start with a tolerance of 0.1 hours
and increase until we have at least 6 points and at least one supercar factory.                                    
                                                                                                                   
Step 6: Combine the points.                                                                                        
                                                                                                                   
Step 7: Create a DataFrame and plot the map.                                                                       
                                                                                                                   
Let's start with step 1.                                                                                           
                                                                                                                   
Note: We must be cautious because the web_agent returns a string? But the tool says: "Browses the web to find      
information" and returns a string. However, we asked for a list of dictionaries. We must hope that the web_agent   
returns a structured [38;2;2

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
                Here is your code snippet:
                <think>
We are going to follow the plan step by step.

Step 1: Use the web_agent to get Batman filming locations with coordinates.
   - We'll ask the web_agent to return a list of dictionaries with keys: 'name', 'latitude', 'longitude'

Step 2: For each Batman location, calculate the travel time from Gotham (40.7128, -74.0060) using 
`calculate_cargo_travel_time`.

Step 3: Use the web_agent to get supercar factory locations with coordinates.

Step 4: For each supercar factory, calculate the travel time from Gotham.

Step 5: Match supercar factories to Batman travel times with a tolerance. We'll start with a tolerance of 0.1 hours
and increase until we have at least 6 points and at least one supercar factory.

Step 6: Combine the points.

Step 7: Create a DataFrame and plot the map.

Let's start with step 1.

Note: We must be cautious because the web_agent returns a string? But the tool says: "Browses the web to find 
information" and returns a string. However, we asked for a list of dictionaries. We must hope that the web_agent 
returns a structured list? But the tool doesn't specify. We might have to parse the string. Alternatively, we can 
ask the web_agent to return in a specific format (like JSON) so we can parse it.

But the tool doesn't specify the output format. We'll assume the web_agent returns a string that we can parse. 
Alternatively, we can ask the web_agent to return JSON.

We'll do:

For step 1: 
   task = "Find a comprehensive list of filming locations for Batman movies (the entire franchise) around the 
world. For each location, provide the name of the place, city, country, and the latitude and longitude. If the 
coordinates are not provided, try to get them by geocoding the address. Return the list as a JSON string of a list 
of dictionaries, each with keys: 'name', 'latitude', 'longitude'."

Similarly for step 3.

But note: the web_agent might not return JSON. We might have to parse a string. We'll try to get it in JSON and 
then use `json.loads` to parse.

Alternatively, if the web_agent returns a string that is not JSON, we might have to extract the information by 
printing and then manually parsing? But that would be too hard. We'll rely on the web_agent to return a structured 
JSON string.

Let's code step 1.

We'll use:
   batman_locations_str = web_agent(task=task1, additional_args={})
   Then parse it to a list of dicts.

But note: the web_agent might return an error or not find any. We'll have to handle that.

We'll do the same for supercar factories.

Let's write the code for step 1 and 2.

Note: We are going to write the code in multiple code blocks because we cannot chain too many tool calls in one 
block? Actually, the web_agent call is one tool call, and then we do the travel time calculations which are 
multiple tool calls. But the travel time function is deterministic and fast, so we can do in one block.

But note: the rule says for tools without JSON output schema, we should not chain too many. The web_agent does not 
have a JSON output schema? We don't know. So we'll do:

   Step 1: Call web_agent for Batman locations and print the result.
   Then in the next step, we parse the result and compute travel times.

But we can do step 1 and 2 in two separate code blocks? Actually, we can do step 1 in one block and then step 2 in 
the next.

Alternatively, we can do step 1 and then in the same block, if the result is a JSON string, parse it and then 
compute the travel times? But that would be one block with one web_agent call and then multiple travel time calls. 
That is acceptable because the travel time function is not a tool that we are calling via the same mechanism? 
Actually, the travel time function is a local function. So we can do.

But note: the web_agent call might take time and we don't want to time out. We'll

...

Check check_reasoning_and_plot failed with error: The api_key client option must be set either by passing api_key 
to the client or by setting the OPENAI_API_KEY environment variable

[Step 15: Duration 27.26 seconds| Input tokens: 215,689 | Output tokens: 10,140]

Reached max steps.

[Step 16: Duration 12.25 seconds| Input tokens: 241,275 | Output tokens: 10,720]

'<think>\nWe have successfully calculated and saved the data to a CSV file. Since we were unable to create the map visualization due to import restrictions, we\'ll return the CSV file path as the final answer. The CSV contains all the required information: locations, coordinates, travel times, and types (Batman locations or supercar factories).\n\nThe solution includes:\n- 12 Batman filming locations worldwide\n- 8 supercar factories with travel times matching Batman locations\n- Total of 20 points (exceeding the minimum requirement of 6)\n- Travel times calculated from Gotham (40.7128° N, 74.0060° W)\n\nFinal Answer:\n```json\n{\n  "status": "success",\n  "message": "Data saved to CSV file",\n  "csv_file": "locations_data.csv",\n  "content": "The file contains all Batman filming locations and matching supercar factories with their coordinates and travel times from Gotham."\n}\n```\n</think>\nI\'ve completed all calculations and saved the data, but encountered limitations in creating t

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)